# cppyy tutorial
#### all credit to the author, Wim Lavrijsen
#### bugs and feature requests at https://bitbucket.org/wlav/cppyy/issues

![title](./cpp_python_feature_mapping.png)

In [ ]:
from __future__ import print_function
p = print
import cppyy

## Define a class

In [ ]:
cppyy.cppdef("""
  struct Integer {
    Integer(int i) : num(i) {}
    int num;
    private:
    int p_num = -1;
  };
""")

In [ ]:
from cppyy.gbl import Integer
m1 = Integer(42)

p(m1.num)
try:
    p(m1.p_num)
except:
    p('p_num not available')

### Autogenerated help/doc

In [ ]:
Integer?

In [ ]:
help(Integer)

## Dynamically add class methods

In [ ]:
cppyy.cppdef("""
    Integer add(Integer &m1, Integer &m2) {
        return Integer(m1.num + m2.num);
    }
""")
Integer.add = cppyy.gbl.add

In [ ]:
m2 = Integer(58)
p(m1.add(m2).num)

## Inheritance

In [ ]:
cppyy.cppdef("""
   struct Base {
       virtual std::string method() { return "Base"; }
   };
   
   struct Derived : Base {
       std::string method() override { return "Derived"; }
   };""")
Base, Derived = cppyy.gbl.Base, cppyy.gbl.Derived

In [ ]:
p(Base().method())
p(Derived().method())

## Operator overloading

In [ ]:
cppyy.cppdef("""
    struct IntValue {
        IntValue(int v) : num(v) {}
        IntValue &operator+(const IntValue &i) { num += i.num; return *this; }
        IntValue &operator+(int i) { num += i; return *this; }
        int num;
    };""")
IntValue = cppyy.gbl.IntValue

In [ ]:
IntValue.__repr__ = lambda self : str(self.num)

p(IntValue(1) + IntValue(1))
p(IntValue(3) + 7)
x, y, z = IntValue(1), IntValue(2), IntValue(3)
p(x + y + z)

## Template classes

In [ ]:
vector = cppyy.gbl.vector
v = vector(int)((0, 8, 42)) # vector[int] and vector("int") also allowed
print(v[0], v[1], v[2])

In [ ]:
cppyy.cppdef("""
template <typename T, int N>
struct fac
{
    static constexpr int value = N * fac<T, N-1>::value;
};

template <typename T>
struct fac<T, 0>
{
    static constexpr int value = 1;
};
""")
from cppyy.gbl import fac

In [ ]:
print(fac(int, 5).value)

## Function overloading

In [ ]:
cppyy.cppdef("""
double dub(double x) { return x*2; }

int dub(int x) { return x*2; }

template <typename T>
T dub(T t) { return t + t; }
""")
dub = cppyy.gbl.dub

In [ ]:
type(dub(3))

In [ ]:
type(dub(3.))

In [ ]:
i = IntValue(2)
i2 = dub(i)
print(i2, '---', type(i2))

## Template methods

In [ ]:
cppyy.cppdef("""
struct Doubler {
    int operator()(int t) { return t + t; } // at least one overload must be non-template

    template <typename T>
    T operator()(T t) { return t + t; }
};
""")
from cppyy.gbl import Doubler

In [ ]:
dub = Doubler()
six = dub(IntValue(3))
print(six, '---', type(six))

## Lambdas

In [5]:
cppyy.cppdef("""
auto make_lambda() {
    int b = 40;
    return [=](int a) { return b+a; };
}
auto l = make_lambda();
""")
print(cppyy.gbl.l(2))

42


## Including a header

In [ ]:
f = open('/tmp/cppyy_test.h','w') 
f.write('std::string bar() { return "oh yeah"; }')
f.close() 
cppyy.include('/tmp/cppyy_test.h')

In [ ]:
p(cppyy.gbl.bar())

## Automatic creation of bindings with ROOT

[Bindings with ROOT](root_make_bindings.ipynb)

## TTree to numpy: pythonizations

custom python module augments ROOT classes when imported: `$ROOTSYS/ROOT.py`, e.g.:

```python
# a python function that retrieves large datasets via a C++ helper, then converts to numpy array
def _TTreeAsMatrix():
    ...
    pass

# actual type augmentation
_root.CreateScopeProxy('TTree').AsMatrix = _TTreeAsMatrix
```

## Calling python from C++ from python

[simple](root_call_python_simple.ipynb)

[build_menu](build_menu.ipynb)

[dynamic_canvas](dynamic_canvas.ipynb)

## Inheriting from C++ classes in python

[Tutorial here](http://nbviewer.jupyter.org/url/root.cern.ch/notebooks/HowTos/inherit-C++-classes-from-python.ipynb)

## Automatic creation of bindings from a cmake project

In [ ]:
# work in progress, test repo is at https://github.com/bluehood/cppyy_cmake_bindings